### LSL Implementation

In [ ]:
import numpy as np
import pandas as pd
import serial
import time
from pylsl import StreamInlet, resolve_stream

def process_chunk(chunk):
    if chunk:
        # Extract EEG data from the chunk (assuming columns 3 to 16 contain EEG data)
        eeg_data = [row[3:17] for row in chunk]
        eeg_data = np.array(eeg_data)
        
        # Create a DataFrame from the transposed EEG data
        EEG_df_whole = pd.DataFrame(eeg_data.T)
        
        # Define the row names (channel names)
        channel_names = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']
        
        # Set the row names
        EEG_df_whole.index = channel_names
        
        # Access the first row (AF3)
        first_row = EEG_df_whole.iloc[0]
        print("First row data (AF3):")
        print(first_row)
        
        # Transmit the first row data serially
        transmit_data_serially(first_row)

def transmit_data_serially(data):
    """
    Transmits data serially to a virtual serial port.
    
    Parameters:
    data (Series): The row data to be transmitted.
    """
    # Set up the serial port
    serial_port = '/dev/ttys001'  # Replace with your virtual serial port
    baud_rate = 9600  # Set according to your needs
    
    # Open the serial port
    ser = serial.Serial(serial_port, baud_rate)
    
    # Transmit each data point in the row
    for value in data:
        ser.write((str(value) + '\n').encode('utf-8'))  # Convert the data to string, append newline, and encode
        print(f"Sent: {value}")
        time.sleep(0.00833)  # Approximate time to transmit one data point
    
    # Close the serial port
    ser.close()

def main():
    print("Looking for an EEG stream...")
    streams = resolve_stream('type', 'EEG')
    inlet = StreamInlet(streams[0])

    while True:
        chunk, timestamps = inlet.pull_chunk(timeout=2.0)
        if not chunk:
            print("No data received. Stopping transmission.")
            break  # Exit the loop if no new data is received within the timeout period
        process_chunk(chunk)

if __name__ == "__main__":
    main()


In [2]:
import numpy as np
import pandas as pd
import serial
import time
from pylsl import StreamInlet, resolve_stream
from scipy.signal import welch

def compute_bandpower(data, sf, band, window_sec=4):
    """Compute the average power of the signal x in a specific frequency band.
    
    Parameters
    ----------
    data : 1d-array
        Input signal in the time-domain.
    sf : float
        Sampling frequency of the data.
    band : list
        Lower and upper frequencies of the band of interest.
    window_sec : float
        Length of each window in seconds.
    
    Returns
    -------
    bp : float
        Band power.
    """
    band = np.asarray(band)
    low, high = band
    
    nperseg = int(window_sec * sf)
    freqs, psd = welch(data, sf, nperseg=nperseg)
    
    idx_band = np.logical_and(freqs >= low, freqs <= high)
    bp = np.mean(psd[idx_band])
    return bp

def process_chunk(chunk, sf):
    if chunk:
        # Extract EEG data from the chunk (assuming columns 3 to 16 contain EEG data)
        eeg_data = [row[3:17] for row in chunk]
        eeg_data = np.array(eeg_data)
        
        # Create a DataFrame from the transposed EEG data
        EEG_df_whole = pd.DataFrame(eeg_data.T)
        
        # Define the row names (channel names)
        channel_names = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']
        
        # Set the row names
        EEG_df_whole.index = channel_names
        
        # Access the data for AF3 and AF4
        af3_data = EEG_df_whole.loc['AF3'].values
        af4_data = EEG_df_whole.loc['AF4'].values

        # Compute the alpha band power (8-12 Hz) for AF3 and AF4
        alpha_band = [8, 12]
        af3_bandpower = compute_bandpower(af3_data, sf, alpha_band)
        af4_bandpower = compute_bandpower(af4_data, sf, alpha_band)
        
        # Average the bandpowers
        avg_bandpower = (af3_bandpower + af4_bandpower) / 2
        print("Average Alpha Bandpower (AF3 and AF4):", avg_bandpower)
        
        # Transmit the average bandpower serially
        transmit_data_serially(avg_bandpower)

def transmit_data_serially(data):
    """
    Transmits data serially to a virtual serial port.
    
    Parameters:
    data (float): The data to be transmitted.
    """
    # Set up the serial port
    serial_port = '/dev/ttys001'  # Replace with your virtual serial port
    baud_rate = 9600  # Set according to your needs
    
    # Open the serial port
    ser = serial.Serial(serial_port, baud_rate)
    
    # Transmit the data
    ser.write((str(data) + '\n').encode('utf-8'))  # Convert the data to string, append newline, and encode
    print(f"Sent: {data}")
    
    # Close the serial port
    ser.close()

def main():
    print("Looking for an EEG stream...")
    streams = resolve_stream('type', 'EEG')
    inlet = StreamInlet(streams[0])

    # Assuming a sample rate of 256 Hz (adjust as necessary)
    sf = 256

    while True:
        chunk, timestamps = inlet.pull_chunk(timeout=2.0)
        if not chunk:
            print("No data received. Stopping transmission.")
            break  # Exit the loop if no new data is received within the timeout period
        process_chunk(chunk, sf)

if __name__ == "__main__":
    main()


Looking for an EEG stream...


/Users/apple/anaconda3/envs/mne/lib/python3.11/site-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 1024 is greater than input length  = 512, using nperseg = 512
  warnings.warn('nperseg = {0:d} is greater than input length '


Average Alpha Bandpower (AF3 and AF4): 151.52372439812692
Sent: 151.52372439812692
Average Alpha Bandpower (AF3 and AF4): 59.394103743167825
Sent: 59.394103743167825
Average Alpha Bandpower (AF3 and AF4): 168.39411551621114
Sent: 168.39411551621114
Average Alpha Bandpower (AF3 and AF4): 56.227561675823004
Sent: 56.227561675823004
Average Alpha Bandpower (AF3 and AF4): 42.57002215210239
Sent: 42.57002215210239
Average Alpha Bandpower (AF3 and AF4): 39.071852934581194
Sent: 39.071852934581194
Average Alpha Bandpower (AF3 and AF4): 69.6411953459404
Sent: 69.6411953459404
Average Alpha Bandpower (AF3 and AF4): 88.85767178456027
Sent: 88.85767178456027
Average Alpha Bandpower (AF3 and AF4): 88.8202893966537
Sent: 88.8202893966537
Average Alpha Bandpower (AF3 and AF4): 90.14869491727404
Sent: 90.14869491727404
Average Alpha Bandpower (AF3 and AF4): 108.53923619169362
Sent: 108.53923619169362
Average Alpha Bandpower (AF3 and AF4): 48.675518493996464
Sent: 48.675518493996464
Average Alpha Band

/Users/apple/anaconda3/envs/mne/lib/python3.11/site-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 1024 is greater than input length  = 768, using nperseg = 768
  warnings.warn('nperseg = {0:d} is greater than input length '


Average Alpha Bandpower (AF3 and AF4): 431.2726229308982
Sent: 431.2726229308982
Average Alpha Bandpower (AF3 and AF4): 568.6886780204348
Sent: 568.6886780204348
Average Alpha Bandpower (AF3 and AF4): 376.01038059471114
Sent: 376.01038059471114
Average Alpha Bandpower (AF3 and AF4): 304.83545524705994
Sent: 304.83545524705994
Average Alpha Bandpower (AF3 and AF4): 349.9100810247144
Sent: 349.9100810247144
Average Alpha Bandpower (AF3 and AF4): 53340.94666202876
Sent: 53340.94666202876
Average Alpha Bandpower (AF3 and AF4): 55420.598335193856
Sent: 55420.598335193856
Average Alpha Bandpower (AF3 and AF4): 37612.54928614904
Sent: 37612.54928614904
Average Alpha Bandpower (AF3 and AF4): 33835.80305736727
Sent: 33835.80305736727
Average Alpha Bandpower (AF3 and AF4): 496.44768370041095
Sent: 496.44768370041095
Average Alpha Bandpower (AF3 and AF4): 8793.714643002684
Sent: 8793.714643002684
Average Alpha Bandpower (AF3 and AF4): 142523.68676486635
Sent: 142523.68676486635
Average Alpha Band

...

/Users/apple/anaconda3/envs/mne/lib/python3.11/site-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 1024 is greater than input length  = 618, using nperseg = 618
  warnings.warn('nperseg = {0:d} is greater than input length '


Average Alpha Bandpower (AF3 and AF4): 7.381852782775324
Sent: 7.381852782775324


/Users/apple/anaconda3/envs/mne/lib/python3.11/site-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 1024 is greater than input length  = 406, using nperseg = 406
  warnings.warn('nperseg = {0:d} is greater than input length '


Average Alpha Bandpower (AF3 and AF4): 3.436591748458608
Sent: 3.436591748458608
Average Alpha Bandpower (AF3 and AF4): 6.080420811186611
Sent: 6.080420811186611
Average Alpha Bandpower (AF3 and AF4): 23.272709762566166
Sent: 23.272709762566166
Average Alpha Bandpower (AF3 and AF4): 5.803576691499776
Sent: 5.803576691499776
Average Alpha Bandpower (AF3 and AF4): 41.210883739145586
Sent: 41.210883739145586
Average Alpha Bandpower (AF3 and AF4): 19.374872878288144
Sent: 19.374872878288144
Average Alpha Bandpower (AF3 and AF4): 6.576685067531667
Sent: 6.576685067531667
Average Alpha Bandpower (AF3 and AF4): 4.298618720608496
Sent: 4.298618720608496
Average Alpha Bandpower (AF3 and AF4): 28.072525611084515
Sent: 28.072525611084515
Average Alpha Bandpower (AF3 and AF4): 39.368664198885554
Sent: 39.368664198885554
Average Alpha Bandpower (AF3 and AF4): 47.85823280553332
Sent: 47.85823280553332
Average Alpha Bandpower (AF3 and AF4): 437.63445811045074
Sent: 437.63445811045074
Average Alpha Ba

2024-07-15 10:47:13.588 (2769.235s) [R_EmotivDataSt  ]      data_receiver.cpp:344    ERR| Stream transmission broke off (Input stream error.); re-connecting...
